In [1]:

# wget  -P $FLINK_HOME/lib https://repo1.maven.org/maven2/org/apache/flink/flink-parquet/1.15.3/flink-parquet-1.15.3.jar
# wget  -P $FLINK_HOME/lib https://repo1.maven.org/maven2/org/apache/flink/flink-shaded-hadoop2-uber/2.7.5-1.8.3/flink-shaded-hadoop2-uber-2.7.5-1.8.3.jar

# wget  -P $FLINK_HOME/lib https://repository.cloudera.com/artifactory/cloudera-repos/org/apache/flink/flink-shaded-hadoop-3-uber/3.1.1.7.2.8.0-224-9.0/flink-shaded-hadoop-3-uber-3.1.1.7.2.8.0-224-9.0.jar



# Python location when installed via pip install
# /home/krish/flink-dev/lib/python3.8/site-packages/pyflink/lib


In [2]:
import datetime
from pyflink.table import EnvironmentSettings, TableEnvironment, DataTypes, CsvTableSource

from pyflink.table.expressions import col
from pyflink.table.window import Over, GroupWindow
from pyflink.table.expressions import col, UNBOUNDED_RANGE, CURRENT_RANGE
from pyflink.table.udf import udf
# create a batch TableEnvironment
env_settings = EnvironmentSettings.in_batch_mode()
table_env = TableEnvironment.create(env_settings)


table_env.get_config().get_configuration().set_string("pipeline.jars", "file:///opt/flink-1.15.3/lib/flink-shaded-hadoop-3-uber-3.1.1.7.2.8.0-224-9.0.jar;file:///opt/flink-1.15.3/lib/flink-parquet-1.15.3.jar")


In [3]:
# InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
# 536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 8:26,2.55,17850,United Kingdom

source_ddl = """
            CREATE TABLE invoices (
                   InvoiceNo STRING,
                   StockCode  STRING,
                   Description  STRING,
                   Quantity DOUBLE,
                   InvoiceDate TIMESTAMP(3),
                   UnitPrice DOUBLE,
                   CustomerID STRING,
                   Country STRING,
                   WATERMARK FOR InvoiceDate AS InvoiceDate - INTERVAL '1' SECOND

                    ) WITH (
                      'connector' = 'filesystem',          
                     'path' = 'file:////home/krish/flink-dev/flink/data/ecommerce-clean.csv', 
                      'format' = 'csv'
                    )"""


table_env.execute_sql(source_ddl) 

invoices = table_env.from_path("invoices")
##############################
print('\nRegistered Tables List')
print(table_env.list_tables())

print('\nFinancial Trxs Schema')
invoices.print_schema()


Registered Tables List
['invoices']

Financial Trxs Schema
(
  `InvoiceNo` STRING,
  `StockCode` STRING,
  `Description` STRING,
  `Quantity` DOUBLE,
  `InvoiceDate` TIMESTAMP(3),
  `UnitPrice` DOUBLE,
  `CustomerID` STRING,
  `Country` STRING,
  WATERMARK FOR `InvoiceDate`: TIMESTAMP(3) AS `InvoiceDate` - INTERVAL '1' SECOND
)


In [4]:
invoices.fetch(3).execute().print()

+--------------------------------+--------------------------------+--------------------------------+--------------------------------+-------------------------+--------------------------------+--------------------------------+--------------------------------+
|                      InvoiceNo |                      StockCode |                    Description |                       Quantity |             InvoiceDate |                      UnitPrice |                     CustomerID |                        Country |
+--------------------------------+--------------------------------+--------------------------------+--------------------------------+-------------------------+--------------------------------+--------------------------------+--------------------------------+
|                         536365 |                         85123A | WHITE HANGING HEART T-LIGHT... |                            6.0 | 2010-12-01 08:26:00.000 |                           2.55 |                          17850

In [5]:
# sink to write parquet files
# file:///home/rps/workshop/data/ecommerce-clean.parquet is a folder, will contain many partitions files
sink_ddl_parquet=f"""
    create table `EComParquetResult`(
                 InvoiceNo STRING,
                   StockCode  STRING,
                   Description  STRING,
                   Quantity DOUBLE,
                   InvoiceDate TIMESTAMP(3),
                   UnitPrice DOUBLE,
                   CustomerID STRING,
                   Country STRING
    ) with (
        'connector' = 'filesystem',
        'format' = 'parquet',
        'path' = 'file:////home/krish/flink-dev/flink/data/ecommerce-clean.parquet'
    )
    """

table_env.execute_sql(sink_ddl_parquet)

In [6]:
# emit the data from source to target

table_env.execute_sql("INSERT INTO EComParquetResult select * from invoices").wait()

invoices.execute_insert("EComParquetResult")

Py4JJavaError: An error occurred while calling o1.executeSql.
: java.lang.NoClassDefFoundError: org/apache/parquet/hadoop/ParquetWriter$Builder
	at java.lang.ClassLoader.defineClass1(Native Method)
	at java.lang.ClassLoader.defineClass(ClassLoader.java:756)
	at java.security.SecureClassLoader.defineClass(SecureClassLoader.java:142)
	at java.net.URLClassLoader.defineClass(URLClassLoader.java:473)
	at java.net.URLClassLoader.access$100(URLClassLoader.java:74)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:369)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:363)
	at java.security.AccessController.doPrivileged(Native Method)
	at java.net.URLClassLoader.findClass(URLClassLoader.java:362)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:352)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.flink.formats.parquet.ParquetFileFormatFactory$1.createRuntimeEncoder(ParquetFileFormatFactory.ja